# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/ana/projects/mine/data-eng/data_modeling/cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data


### Query 1 -  Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### Create Table

In [8]:
query = "CREATE TABLE IF NOT EXISTS artist_and_song "
query = query + '''(session_id int,
                    item_session int, 
                    artist varchar, 
                    song varchar, 
                    song_length double, 
                    PRIMARY KEY(session_id, item_session))'''
try:
    session.execute(query)
except Exception as e:
    print(e)                    

#### Insert into Table

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_and_song (session_id, item_session, artist, song, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Select the respective query

In [10]:
query = "select artist, song, song_length from artist_and_song WHERE session_id=338 and item_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


#### Query Description

        In our dataset, we need to query the artist, song and its length with the 
        filter SessionId 338 and the itemInSession 4. This is the result:
    
| Artist    | Song                            | Song's Length |
|-----------|---------------------------------|---------------|
| Faithless | Music Matters (Mark Knight Dub) | 495.3073      |

### Query 2 - Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Create Table

In [11]:
query = "CREATE TABLE IF NOT EXISTS song_and_user "
query = query + '''(artist varchar, 
                    song varchar, 
                    first_name varchar,
                    last_name varchar,
                    user_id int,
                    session_id int,
                    item_session int, 
                    PRIMARY KEY((user_id, session_id), item_session))
                    WITH CLUSTERING ORDER BY (item_session ASC);
                    '''
try:
    session.execute(query)
except Exception as e:
    print(e)         

#### Insert into Table

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_and_user (artist, song, first_name, last_name, user_id, session_id, item_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))

#### Select the respective query

In [13]:
query = "select artist, song, first_name, last_name from song_and_user WHERE user_id=10 and session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.first_name, row.last_name)

                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Query Description

                In our dataset, we need to query name of artist, song (sorted by itemInSession) 
                and user(first and last name) for userid = 10, sessionid = 182. This is the result:

| Artist            | Song(sorted by ItemInSession)                        | User(first and last name) |
|-------------------|------------------------------------------------------|---------------------------|
| Down To The Bone  | Keep On Keepin' On                                   | Sylvie Cruz               |
| Three Drives      | Greece 2000                                          | Sylvie Cruz               |
| Sebastien Tellier | Kilometer                                            | Sylvie Cruz               |
| Lonnie Gordon     | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie Cruz               |

### Query 3 -  Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Create Table

In [14]:
query = "CREATE TABLE IF NOT EXISTS user_by_song "
query = query + '''(first_name varchar,
                    last_name varchar,
                    user_id int,
                    song varchar, 
                    PRIMARY KEY(song, user_id));
                    '''
try:
    session.execute(query)
except Exception as e:
    print(e)         

#### Insert into Table

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_by_song (first_name, last_name, user_id, song)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[1], line[4], int(line[10]), line[9]))

#### Select the respective query

In [16]:
query = "select first_name, last_name from user_by_song WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


#### Query Description

                In our dataset, we need to query user's first and last name
                for song title 'All Hands Against His Own'. This is the result:

| First Name | Last Name |
|------------|-----------|
| Jacqueline | Lynch     |
| Tegan      | Levine    |
| Sara       | Johnson   |

### Drop the tables before closing out the sessions

In [17]:
query1 = "DROP TABLE artist_and_song"
query2 = "DROP TABLE song_and_user"
query3 = "DROP TABLE user_by_song"
try:
    session.execute(query1)
except Exception as e:
    print(e)  
try:
    session.execute(query2)
except Exception as e:
    print(e) 
try:
    session.execute(query3)
except Exception as e:
    print(e) 

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()